In [1]:
import sys

sys.path.append("/Users/shinewu/codes/xiang_wu_experimental/xiang-wu-experimental")

In [2]:
import collections
import csv
import itertools
import matplotlib.pyplot as plt
import json
import os
import pathlib
import re

import pandas as pd

from python import utils, word_matchers, word_models

In [3]:
print(word_matchers.extract_words("aa12 Bc! #Net$"))
print(word_matchers.split_at_capital("SuperCoolNFT"))
print(word_matchers.tokenize("moneyTake123 #BadApple$$ $N$"))

['aa', '12', 'Bc', 'Net', '$']
['Super', 'Cool', 'NFT']
['money', 'take', '123', 'bad', 'apple', '$$', 'n', '$']


In [4]:
MIN_30D_SALES = 1

ALL_COLLECTION_SQL = f"""
SELECT
  a.ID,
  a.NAME,
  a.DEFAULT_CHAIN_ID,
  a.SLUG,
  a.DESCRIPTION
FROM
  PC_FIVETRAN_DB.PUBLIC.API_COLLECTION a
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_COLLECTIONSTATS b
  ON a.ID = b.COLLECTION_ID
WHERE
  a.IS_DELISTED = FALSE
  AND a.HIDDEN = FALSE
  AND a.IS_NSFW = FALSE
  AND a.BLACKLISTED = FALSE
  AND b.THIRTY_DAY_SALES >= {MIN_30D_SALES}
"""

print(ALL_COLLECTION_SQL)


SELECT
  a.ID,
  a.NAME,
  a.DEFAULT_CHAIN_ID,
  a.SLUG,
  a.DESCRIPTION
FROM
  PC_FIVETRAN_DB.PUBLIC.API_COLLECTION a
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_COLLECTIONSTATS b
  ON a.ID = b.COLLECTION_ID
WHERE
  a.IS_DELISTED = FALSE
  AND a.HIDDEN = FALSE
  AND a.IS_NSFW = FALSE
  AND a.BLACKLISTED = FALSE
  AND b.THIRTY_DAY_SALES >= 1



In [5]:
RUN_SQL = False
ALL_COLLECTION_DF_FILE = pathlib.Path(f"/Users/shinewu/tmp/all_collection_min_30d_sales_{MIN_30D_SALES}.pickle")

if RUN_SQL:
    ALL_COLLECTION_DF = utils.snowflake_run_query(ALL_COLLECTION_SQL)
    ALL_COLLECTION_DF.to_pickle(ALL_COLLECTION_DF_FILE)
else:
    ALL_COLLECTION_DF = pd.read_pickle(ALL_COLLECTION_DF_FILE)

In [6]:
ALL_COLLECTION_DF.iloc[100:120]

,ID,NAME,DEFAULT_CHAIN_ID,SLUG,DESCRIPTION
100,1924641,NiftyCookies,1,niftycookies,Is this art?
101,14579724,Hawaii Shot on Film,1,hawaii-shot-on-film,"The ""Hawaii Shot on Film"" is a 1/1 collection ..."
102,1290331,Lemaitre Series,1,lemaitre,"Evolution of an artist, this is an ever evolvi..."
103,4134603,"The Starry Night, 1889 by Vincent Van Gogh",3,the-starry-night-1889-by-vincent-van-gogh,"The Starry Night, 1889 by Vincent Van Gogh\nWi..."
104,675375,My Brother Is A Jerk.....,1,my-brother-is-a-jerk,Since I was a kid... my brother has been a jer...
105,14495850,Official Youtopia,1,official-youtopia,Youtopia is based on the concept of Web 3 Meta...
106,8466410,Crypto octupus,1,cryptooctupus,"Welcome to my ""Pulpitante"" collection"
107,14221798,PI - g9DkUvMr9B,3,pi-g9dkuvmr9b,None
108,1516023,KlayChicken V1,2,klaychicken,### The Very First KLAYPROJECT - KlayChicken\n...
109,6298791,JiggyVerse,1,jiggyverse,"The JiggyVerse is an alternate universe of 9,4..."


In [7]:
list(word_matchers.match_all(word_models.ACTIVE_GET_PAYMENT_MATCHER,
               word_matchers.tokenize("paying you no dollar and you get paid $$$ and you receive monthly payment")))

[(6, 9), (11, 14)]

In [8]:
word_models.FINANCIAL_FREEDOM_MATCHER.match(0, word_matchers.tokenize("financial total freedom!"))

(0, 3)

In [9]:
word_models.RETIRE_EARLY_MATCHER.match(0, word_matchers.tokenize("Retires pretty sure early!"))

(0, 4)

In [10]:
word_models.PASSIVE_TRADE_SECURITY_MATCHER.match(0, word_matchers.tokenize("stocks will be sold to him"))

(0, 4)

In [11]:
def label_with_matcher(row):
    all_text = str(row["NAME"]) + " " + str(row["DESCRIPTION"])
    all_tokens = word_matchers.tokenize(all_text)


    for label, matcher in [
        ("ACTIVE_GET", word_models.ACTIVE_GET_PAYMENT_MATCHER),
        ("PASSIVE_GET", word_models.PASSIVE_GET_PAYMENT_MATCHER),
        ("FINANCIAL_FREEDOM", word_models.FINANCIAL_FREEDOM_MATCHER),
        ("RETIRE_EARLY", word_models.RETIRE_EARLY_MATCHER),
        ("FINANCIAL_INSTRUMENTS", word_models.FINANCIAL_INSTRUMENTS_MATCHER),
        ("ACTIVE_TRADE", word_models.ACTIVE_TRADE_SECURITY_MATCHER),
        ("PASSIVE_TRADE", word_models.PASSIVE_TRADE_SECURITY_MATCHER),
    ]:
        matches = list(word_matchers.match_all(matcher, all_tokens))
        if matches:
            return label, all_tokens[matches[0][0]:matches[0][1]]
    
    return "SAFE", []


LABELS_WITH_MATCHER = ALL_COLLECTION_DF.apply(label_with_matcher, axis=1)

In [12]:
desc = '''


Aiyah Verse Stickman-X Genesis 465 out of 666 Genesis Stickman-X have awakened on the Ethereum ERC721 blockchain. Nobody know when the rest will do so. These Stickman-X are the origins of the vast Aiyah Verse and will allow their masters to travel between the worlds within. New worlds are uncovered as time moves forward... to explore the new or to dominate the old, their fate in the Aiyah Verse is yours to decide.

Holders of the Stickman-X Genesis will be entitled to profit-sharing in our upcoming mobile game - Crypto Warframe as well as secondary sale revenue-sharing in our upcoming ID NFTs to be used in the game


'''

print(list(word_matchers.match_all(word_models.FINANCIAL_INSTRUMENTS_MATCHER, word_matchers.tokenize(desc))))

[(87, 89), (101, 103)]


In [16]:
num_printed = 0
for row, (label, matched) in zip(ALL_COLLECTION_DF.itertuples(), LABELS_WITH_MATCHER):
    if label != "SAFE":
        if num_printed >= 400 and num_printed < 500:
            print(label, matched, utils.slug_to_url(row.SLUG))
            print(row.NAME, row.DESCRIPTION)
            print("==========")
        num_printed += 1

print("total =", num_printed)

ACTIVE_GET ['receive', 'income'] https://opensea.io/collection/maisongroupge
Maison Group ###**[NFT+DEFI+REAL ESTATE>>>PRESS ABOUT US - READ MORE](https://koinpost.com/2022/03/04/maison-group-has-created-a-new-trend-in-cryptocurrency-combined-real-estate-nft-and-defi/)**
---
[How to buy on Polygon https://support.opensea.io/hc/en-us/articles/1500012889322-How-do-I-purchase-NFTs-on-Polygon-](https://support.opensea.io/hc/en-us/articles/1500012889322-How-do-I-purchase-NFTs-on-Polygon-)
--
--
Elite residential complex in Batumi (Georgia) at the excavation stage. Housing delivery is scheduled for January 3, 2024. Now you have the option of buying with cryptocurrency in the form of NFT. Become part of a global project. Then send objects to staking with ARPY 37% (go to the Georgian version of the website (only for residents of Georgia) to buy an apartment for lari). Building permission received
You can buy an apartment in whole or in parts and receive income from investments in USDT. 
ACTIVE